<a href="https://colab.research.google.com/github/seyonechithrananda/bert-loves-chemistry/blob/master/MLM_ZINC_250k_ROBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall -y tensorflow
!pip install transformers

In [2]:
!pip install numpy
!pip install pandas

In [11]:
#verify file length

fname = '/content/drive/My Drive/Project De Novo/Molecule Transformer/250k_rndm_zinc_drugs_clean_sorted copy.txt'
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

print(file_len(fname))

249456


In [17]:
%%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer


CPU times: user 18 µs, sys: 1e+03 ns, total: 19 µs
Wall time: 22.4 µs


In [0]:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files='/content/drive/My Drive/Project De Novo/Molecule Transformer/250k_rndm_zinc_drugs_clean_sorted copy.txt', vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

Lets make a new directory, BERT_loves_chemistry, to store our tokenize

In [19]:
%cd /content/drive/My Drive/Project De Novo/Molecule Transformer
!mkdir ZINC_250k_BERT_loves_chemistry
tokenizer.save("ZINC_250k_BERT_loves_chemistry")

/content/drive/My Drive/Project De Novo/Molecule Transformer
mkdir: cannot create directory ‘ZINC_250k_BERT_loves_chemistry’: File exists


['ZINC_250k_BERT_loves_chemistry/vocab.json',
 'ZINC_250k_BERT_loves_chemistry/merges.txt']

In [0]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing 

In [0]:
tokenizer = ByteLevelBPETokenizer(
    "/content/drive/My Drive/Project De Novo/Molecule Transformer/ZINC_250k_BERT_loves_chemistry/vocab.json",
    "/content/drive/My Drive/Project De Novo/Molecule Transformer/ZINC_250k_BERT_loves_chemistry/merges.txt",
)

In [0]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)

In [23]:
#tokenize remdesivir SMILES to test the tokenizer!
tokenizer.encode("CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=C1")


Encoding(num_tokens=76, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [24]:
tokenizer.encode("CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=C1").tokens


['<s>',
 'CCC',
 '(',
 'CC',
 ')',
 'COC',
 '(=',
 'O',
 ')[',
 'C',
 '@',
 'H',
 '](',
 'C',
 ')',
 'N',
 '[',
 'P',
 '@](=',
 'O',
 ')(',
 'OC',
 '[',
 'C',
 '@',
 'H',
 ']',
 '1',
 'O',
 '[',
 'C',
 '@](',
 'C',
 '#',
 'N',
 ')([',
 'C',
 '@',
 'H',
 '](',
 'O',
 ')[',
 'C',
 '@@',
 'H',
 ']',
 '1',
 'O',
 ')',
 'C',
 '1',
 '=',
 'CC',
 '=',
 'C',
 '2',
 'N',
 '1',
 'N',
 '=',
 'CN',
 '=',
 'C',
 '2',
 'N',
 ')',
 'OC',
 '1',
 '=',
 'CC',
 '=',
 'CC',
 '=',
 'C',
 '1',
 '</s>']

## 3. Train a language model from scratch

We will now train our language model using the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) script from `transformers` (newly renamed from `run_lm_finetuning.py` as it now supports training from scratch more seamlessly). Just remember to leave `--model_name_or_path` to `None` to train from scratch vs. from an existing model or checkpoint.

> We’ll train a BERT for chemistry model, with the help of our tokenizdr trained on the ZINC 250k dataset we used.

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.

In [25]:
!nvidia-smi #check GPU 

Mon May 25 04:16:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [26]:
import torch
torch.cuda.is_available() #checking if CUDA + Colab GPU works

True

In [27]:
# Get the example scripts.
!wget -c https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py

--2020-05-25 04:16:31--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



To visualize the training progress, we can use `!tensorboard dev upload --logdir ./path/to/runs`

In [0]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./ZINC_250k_BERT_loves_chemistry", max_len=512)

In [33]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

model.num_parameters()
# => 84 million parameters

84095008

In [34]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./250k_rndm_zinc_drugs_clean_sorted copy.txt",
    block_size=128,
)

CPU times: user 13.5 s, sys: 472 ms, total: 14 s
Wall time: 8.66 s


In [0]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [0]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./ZINC_250k_BERT_loves_chemistry",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [0]:
%%time
trainer.train()

In [0]:
trainer.save_model("./ZINC_250k_BERT_loves_chemistry")



In [38]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="seyonec/ChemBERTa-zinc250k-v1",
    tokenizer="seyonec/ChemBERTa-zinc250k-v1"
)




In [39]:
fill_mask("CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=<mask>1")
#fill_mask("La suno <mask>.")

[{'score': 0.47255268692970276,
  'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=C1</s>',
  'token': 39},
 {'score': 0.0970589816570282,
  'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=21</s>',
  'token': 22},
 {'score': 0.0922168418765068,
  'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=)1</s>',
  'token': 13},
 {'score': 0.08521866798400879,
  'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=(1</s>',
  'token': 12},
 {'score': 0.07716590911149979,
  'sequence': '<s> CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=CC1</s>',
  'token': 262}]